## Predicting sale price
### This notebook contains the necessary code for predicting sale price with ML

Importing packages

In [ ]:
try:
    import pandas as pd
    import plotly.express as px 
    import numpy as np
    import tensorflow as tf
    from sklearn.model_selection import train_test_split
    import sklearn
except:
    !pip install scikit-learn
    !pip install pandas
    !pip install plotly
    !pip install tensorflow
    import pandas as pd
    import plotly.express as px 
    import numpy as np
    import tensorflow as tf
    from sklearn.model_selection import train_test_split
    import sklearn

Defining the modell

In [ ]:
def createModel(normlayer,trainshape1):
    model=tf.keras.Sequential([
        normlayer,
        tf.keras.layers.Dense(256,activation='relu',input_shape=(trainshape1,)),
        tf.keras.layers.Dropout(.1),
        tf.keras.layers.Dense(256,activation='relu'),
        tf.keras.layers.Dropout(.1),
        tf.keras.layers.Dense(128,activation='relu'),
        tf.keras.layers.Dense(1)
        
        
    ])
    model.compile(optimizer='adam',loss='mse',metrics=['mae'])
    return model

Loading data and removing KitchenQual since I have concluded that this knowledge doesn't improve the results as significant as it makes the program more complex 

In [ ]:
data = pd.read_pickle('../data/clean_housing_data.pkl')
a=data.pop('KitchenQual')

In [ ]:
data.columns

Splitting the data to feature and target sets

In [ ]:
features = data.copy()
target=features.pop('SalePrice')

Save the list of important features

In [ ]:
import pickle
with open('../data/important_features.pkl','wb') as f:
    pickle.dump(list(features.columns),f)

Creating test and train sets

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(features,target,test_size=.1)

Setting up the normalisation layer

In [ ]:
normlayer=tf.keras.layers.Normalization()
normlayer.adapt(X_train)

Create and fitting the modell

In [ ]:
es=tf.keras.callbacks.EarlyStopping(patience=10)
model=createModel(normlayer,X_train.shape[1])
model.fit(X_train,Y_train,batch_size=4,epochs=50,verbose=1,validation_split=.2,callbacks=[es])